In [1]:
%load_ext autoreload
%autoreload 2
import os, sys
import warnings
warnings.filterwarnings('ignore') # suppress sklearn deprecation warnings for now..

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# the below checks for whether we run dowhy, causaltune, and FLAML from source
root_path = root_path = os.path.realpath('../..')
try:
    import causaltune
except ModuleNotFoundError:
    sys.path.append(os.path.join(root_path, "causaltune"))

try:
    import dowhy
except ModuleNotFoundError:
    sys.path.append(os.path.join(root_path, "dowhy"))

try:
    import flaml
except ModuleNotFoundError:
    sys.path.append(os.path.join(root_path, "FLAML"))
    
    
    
from causaltune import CausalTune
from causaltune.datasets import synth_ihdp, iv_dgp_econml, generate_non_random_dataset
from causaltune.data_utils import CausalityDataset


OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


In [2]:
# this makes the notebook expand to full width of the browser window
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
df = synth_ihdp(return_df=True).iloc[:,:10]
df['x9'] = '1'
df.loc[0, 'x9'] = '2'
df.loc[1, 'x9'] = '3'
df.loc[2, 'x9'] = '3'
df.loc[3, 'x9'] = '3'
df['y_factual'] = np.random.randint(0, 2, size=len(df))
df.head()

,treatment,y_factual,x1,x2,x3,x4,x5,x6,x7,x8,x9
0,1,1,-0.528603,-0.343455,1.128554,0.161703,-0.316603,1.295216,1,0,2
1,0,1,-1.736945,-1.802002,0.383828,2.244320,-0.629189,1.295216,0,0,3
2,0,0,-0.807451,-0.202946,-0.360898,-0.879606,0.808706,-0.526556,0,0,3
3,0,1,0.390083,0.596582,-1.850350,-0.879606,-0.004017,-0.857787,0,0,3
4,0,1,-1.045229,-0.602710,0.011465,0.161703,0.683672,-0.360940,1,0,1


In [4]:
cd = CausalityDataset(data=df, treatment='treatment', outcomes=['y_factual'])

In [5]:
cd.preprocess_dataset(encoder_type='label')

In [6]:
cd.data

,treatment,y_factual,random,x1,x2,x3,x4,x5,x6,x7,x8,x9
0,1,1,0.0,-0.528603,-0.343455,1.128554,0.161703,-0.316603,1.295216,1.0,0.0,1
1,0,1,0.0,-1.736945,-1.802002,0.383828,2.244319,-0.629189,1.295216,0.0,0.0,2
2,0,0,1.0,-0.807451,-0.202946,-0.360898,-0.879606,0.808706,-0.526556,0.0,0.0,2
3,0,1,1.0,0.390083,0.596582,-1.850350,-0.879606,-0.004017,-0.857787,0.0,0.0,2
4,0,1,0.0,-1.045228,-0.602710,0.011465,0.161703,0.683672,-0.360940,1.0,0.0,3
...,...,...,...,...,...,...,...,...,...,...,...,...
742,0,0,0.0,-0.007654,-0.202946,-0.360898,0.161703,-0.316603,1.792063,0.0,0.0,3
743,1,0,1.0,0.727295,-0.202946,-0.733261,-0.879606,0.808706,1.129600,0.0,0.0,3
744,0,0,0.0,1.181234,0.196818,-1.477987,0.161703,0.746189,0.467138,0.0,0.0,3
745,0,1,0.0,-0.288664,-0.202946,-1.477987,-0.879606,1.621430,0.467138,1.0,0.0,3


In [7]:
2

In [8]:
import pandas as pd
from category_encoders import OneHotEncoder

# Example DataFrame with mixed types (categorical and non-categorical)
data = {
    'x1': ['A', 'B', 'A', 'C'],
    'x2': ['X', 'Y', 'Y', 'Z'],
    'x3': [1, 2, 1, 3],  # Numerical column
    'x4': ['M', 'N', 'M', 'N'],  # Categorical column
    'x5': [True, False, True, True],  # Boolean column
}

# Additional list of groups
groups = ["common_causes", "effect_modifiers", "propensity_modifiers"]

# Example dictionary d with groups as keys and lists of columns as values
d = {
    "common_causes": ["x1", "x2"],
    "effect_modifiers": ["x3"],
    "propensity_modifiers": ["x4", "x5"]
}

df = pd.DataFrame(data)

# Identify categorical columns
categorical_columns = ['x1', 'x4']  # Specify which columns are categorical

# Columns before one-hot encoding
original_columns = df.columns.tolist()

# Apply OneHotEncoder to categorical columns only
encoder = OneHotEncoder(cols=categorical_columns)
df_encoded = encoder.fit_transform(df)

# Columns after one-hot encoding
new_columns = df_encoded.columns.tolist()

# Mapping original columns to new one-hot encoded columns
column_mapping = {}
for original_col in original_columns:
    if original_col in categorical_columns:
        matches = [col for col in new_columns if col.startswith(original_col + '_')]
        if matches:
            column_mapping[original_col] = matches
    else:
        # Non-categorical columns remain the same
        column_mapping[original_col] = [original_col]

# Update dictionary d with new column names
updated_d = {}
for group, columns in d.items():
    updated_columns = []
    for col in columns:
        if col in column_mapping:
            updated_columns.extend(column_mapping[col])
        else:
            updated_columns.append(col)  # If the column wasn't encoded (non-categorical)
    updated_d[group] = updated_columns

# Print the updated dictionary d
print("Updated Dictionary d:")
for group, columns in updated_d.items():
    print(f"{group}: {columns}")

# Display the final encoded DataFrame
print("\nEncoded DataFrame:")
print(df_encoded)


Updated Dictionary d:
common_causes: ['x1_1', 'x1_2', 'x1_3', 'x2']
effect_modifiers: ['x3']
propensity_modifiers: ['x4_1', 'x4_2', 'x5']

Encoded DataFrame:
   x1_1  x1_2  x1_3 x2  x3  x4_1  x4_2     x5
0     1     0     0  X   1     1     0   True
1     0     1     0  Y   2     0     1  False
2     1     0     0  Y   1     1     0   True
3     0     0     1  Z   3     0     1   True
